In [1]:
import numpy as np
from skimage import io
from skimage.transform import resize
import matplotlib.pyplot as plt
import random
import matplotlib.patches as patches
import os

import torch
import torchvision
from torchvision import ops
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

In [ ]:
#To use in Colab
!git clone https://github.com/gabriellecaillaud/APS360_Traffic_Sign_Recognition.git

fatal: destination path 'APS360_Traffic_Sign_Recognition' already exists and is not an empty directory.


In [1]:
import sys
 #To use on colab
sys.path.append('/content/APS360_Traffic_Sign_Recognition')
from RCNN_model.model import *
from RCNN_model.utils import *

In [2]:
from tqdm import tqdm

In [3]:
class ObjectDetectionDataset(Dataset):
    '''
    A Pytorch Dataset class to load the images and their corresponding annotations.
    
    Returns
    ------------
    images: torch.Tensor of size (B, C, H, W)
    gt bboxes: torch.Tensor of size (B, max_objects, 4)
    gt classes: torch.Tensor of size (B, max_objects)
    '''
    def __init__(self, csv_path, img_size, name2idx):
        self.annotation_path = csv_path
        self.img_size = img_size
        self.name2idx = name2idx
        
        self.img_data_all, self.gt_bboxes_all, self.gt_classes_all = self.get_data()
        
    def __len__(self):
        return self.img_data_all.size(dim=0)
    
    def __getitem__(self, idx):
        return self.img_data_all[idx], self.gt_bboxes_all[idx], self.gt_classes_all[idx]
        
    def get_data(self):
        img_data_all = []
        gt_idxs_all = []
        
        gt_boxes_all, gt_classes_all, img_paths = parse_annotation(self.annotation_path, self.img_size)
        
        for i, img_path in tqdm(enumerate(img_paths), total=len(img_paths)):
      
            # skip if the image path is not valid
            if (not img_path) or (not os.path.exists(img_path)):
                continue
            
            # read and resize image
            
            img = io.imread(img_path)
            img = resize(img, self.img_size)
            
            # convert image to torch tensor and reshape it so channels come first
            img_tensor = torch.from_numpy(img).permute(2, 0, 1)
            
            # encode class names as integers
            gt_classes = gt_classes_all[i]
            gt_idx = torch.Tensor([self.name2idx[name] for name in gt_classes])
            
            img_data_all.append(img_tensor)
            gt_idxs_all.append(gt_idx)
        
        # pad bounding boxes and classes so they are of the same size

        if len(gt_boxes_all)!=0 and len(gt_idxs_all)!=0 :
          
          gt_bboxes_pad = pad_sequence(gt_boxes_all, batch_first=True, padding_value=-1)
          gt_classes_pad = pad_sequence(gt_idxs_all, batch_first=True, padding_value=-1)
        
        # stack all images
        print(img_data_all)
        img_data_stacked = torch.stack(img_data_all)[:, :3, :, :]
        
        return img_data_stacked.to(dtype=torch.float32), gt_bboxes_pad, gt_classes_pad

In [5]:
img_width = 640
img_height = 480
csv_path = "/content/APS360_Traffic_Sign_Recognition/dataset_traffic_signs.csv"
image_dir = os.path.join("data", "images")
name2idx = {'pad': -1, '30kmh': 0,'60kmh':1, '100kmh' : 2, 'yield': 3, 'keepRight' :4, 'NoEntry':5, 'NoLeft': 6, 'Stop':7, 'noRight':8, 'ChildrenCrossing' :9 }
idx2name = {v:k for k, v in name2idx.items()}

In [ ]:
import pandas as pd
df = pd.read_csv(csv_path)

In [ ]:
df.drop(3687, inplace=True)

In [ ]:
df.to_csv("df_data.csv")

In [ ]:
df.iloc[3686,:]

Id                                                  3686
imageUrl                       AugData/30kmh/img4207.png
annotation.0.centerX                            0.671296
annotation.0.centerY                            0.647531
annotation.0.width                              0.194444
annotation.0.height                             0.299383
annotation.0.classification                        30kmh
annotation.1.centerX                                 NaN
annotation.1.centerY                                 NaN
annotation.1.width                                   NaN
annotation.1.height                                  NaN
annotation.1.classification                          NaN
Name: 3686, dtype: object

In [ ]:
df.columns

Index(['Id', 'imageUrl', 'annotation.0.centerX', 'annotation.0.centerY',
       'annotation.0.width', 'annotation.0.height',
       'annotation.0.classification', 'annotation.1.centerX',
       'annotation.1.centerY', 'annotation.1.width', 'annotation.1.height',
       'annotation.1.classification'],
      dtype='object')

In [ ]:
len(df)

4606

In [ ]:
od_dataset = ObjectDetectionDataset(csv_path, (img_height, img_width), name2idx)

 33%|███▎      | 1205/3688 [01:51<03:42, 11.18it/s]

In [ ]:
out_c, out_h, out_w = 2048 ,15 ,20


In [ ]:
from model import TwoStageDetector

In [ ]:
img_size = (img_height, img_width)
out_size = (out_h, out_w) ## see other d
n_classes = len(name2idx) - 1 # exclude pad idx
roi_size = (2, 2)

detector = TwoStageDetector(img_size, out_size, out_c, n_classes, roi_size)

In [ ]:
def training_loop(model, learning_rate, train_dataloader, n_epochs):
    
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    model.train()
    loss_list = []
    
    for i in tqdm(range(n_epochs)):
        total_loss = 0
        for img_batch, gt_bboxes_batch, gt_classes_batch in train_dataloader:
            
            # forward pass
            loss = model(img_batch, gt_bboxes_batch, gt_classes_batch)
            
            # backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        loss_list.append(total_loss)
        
    return loss_list